In [1]:
pip install --upgrade fsspec==2024.10.0

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers trl datasets accelerate

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade transformers trl datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from tqdm import tqdm

device = "cuda"

In [6]:
# Загрузка датасета
dataset = load_dataset("esfrankel17/HelpSteer2_binarized")

# Проверка доступных сплитов
print("Доступные сплиты:", list(dataset.keys()))  # ['average_rating_split', ...]

# Выбор основного сплита
main_split = dataset["average_rating_split"]

# Ручное разделение на train/val, если сплит не разделен
if isinstance(main_split, Dataset):
    split_dataset = main_split.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    val_dataset = split_dataset["test"]
else:
    train_dataset = main_split["train"]
    val_dataset = main_split["validation"]

print(f"Размеры: Train={len(train_dataset)}, Val={len(val_dataset)}")

Доступные сплиты: ['average_rating_split', 'average_rating_no_verbosity_split', 'average_rating_no_verbosity_no_complexity_split', 'goodness_score_split']
Размеры: Train=7810, Val=868


In [7]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
sft_model = AutoModelForCausalLM.from_pretrained(sft_model_name)
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
tokenizer.pad_token = tokenizer.eos_token  # Для совместимости с padding

In [8]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
import numpy as np
from tqdm import tqdm

device = "cuda"

# Загрузка датасета
dataset = load_dataset("esfrankel17/HelpSteer2_binarized")["average_rating_split"]

In [9]:
#  Разделение датасета
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Загрязка модели и tokenizer
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
sft_model = AutoModelForCausalLM.from_pretrained(sft_model_name)
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
tokenizer.pad_token = tokenizer.eos_token

# Define RewardModel
class RewardModel(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.reward_head = torch.nn.Linear(base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )
        hidden_states = outputs.hidden_states[-1]
        cls_token = hidden_states[:, 0, :]
        return self.reward_head(cls_token)

reward_model = RewardModel(sft_model)

# Preprocessing function
def preprocess(examples):
    model_inputs = tokenizer(
        examples["prompt"],
        truncation=True,
        padding="max_length",
        max_length=64,
        return_tensors="pt"
    )
    model_inputs["chosen_rating"] = torch.tensor(examples["chosen_rating"], dtype=torch.float)
    return model_inputs

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=16,
    num_train_epochs=1,
    eval_strategy="epoch",
    learning_rate=5e-5,
    fp16=False,
    bf16= True,
    report_to="none",
    save_strategy="no"
)

# Initialize Trainer
trainer = Trainer(
    model=reward_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

Map:   0%|          | 0/6942 [00:00<?, ? examples/s]

Map:   0%|          | 0/1736 [00:00<?, ? examples/s]

/tmp/ipykernel_801239/1039783712.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,No log


TrainOutput(global_step=434, training_loss=-26.511768253168203, metrics={'train_runtime': 43.1328, 'train_samples_per_second': 160.945, 'train_steps_per_second': 10.062, 'total_flos': 0.0, 'train_loss': -26.511768253168203, 'epoch': 1.0})

In [10]:
from safetensors.torch import load_model, save_model

# Сохранение модели
torch.save(trainer.model.state_dict(), "./reward_model_final/pytorch_model.bin")

print('finish')

finish


In [11]:
# Загрузка обученной RM
reward_model.load_state_dict(torch.load("./reward_model_final/pytorch_model.bin"))
reward_model.eval()

RewardModel(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(49152, 576, padding_idx=2)
      (layers): ModuleList(
        (0-29): 30 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=576, out_features=576, bias=False)
            (k_proj): Linear(in_features=576, out_features=192, bias=False)
            (v_proj): Linear(in_features=576, out_features=192, bias=False)
            (o_proj): Linear(in_features=576, out_features=576, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
            (up_proj): Linear(in_features=576, out_features=1536, bias=False)
            (down_proj): Linear(in_features=1536, out_features=576, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        )

In [12]:
from torch.utils.data import DataLoader

def evaluate_reward(model, reward_model, dataset, batch_size=16):
    model.eval()
    total_rewards = []

    # Преобразуем Dataset в формат, совместимый с PyTorch DataLoader
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "chosen_rating"])

    # Создаем DataLoader для батчевой обработки
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluation"):
            # Перемещаем данные на устройство
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Генерация с оптимизацией
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,  # Уменьшаем длину генерации
                do_sample=False,     # Отключаем стохастическую выборку
                pad_token_id=tokenizer.eos_token_id
            )

            # Создаем маску внимания для сгенерированных последовательностей
            gen_attention_mask = (outputs != tokenizer.pad_token_id).to(device)

            # Вычисляем награды
            rewards = reward_model(outputs, gen_attention_mask)
            total_rewards.extend(rewards.cpu().numpy())

    return np.mean(total_rewards)

In [13]:
# Исходная оценка SFT модели
print("Оценка SFT модели до обучения:")
sft_reward = evaluate_reward(sft_model, reward_model, val_dataset)
print(f"Средняя награда: {sft_reward:.3f}")

Оценка SFT модели до обучения:


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Средняя награда: -34.715


In [14]:
from torch.utils.data import DataLoader

def reinforce(sft_model, reward_model, dataset, epochs=1, batch_size=16):
    optimizer = torch.optim.Adam(sft_model.parameters(), lr=1e-5)
    baseline = 0.0
    baseline_alpha = 0.9

    # Установка формата данных для PyTorch
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    # Создание DataLoader с правильным collate_fn
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=lambda batch: {
            'input_ids': torch.stack([x['input_ids'] for x in batch]).to(device),
            'attention_mask': torch.stack([x['attention_mask'] for x in batch]).to(device)
        }
    )

    for epoch in range(epochs):
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            # Генерация ответов
            outputs = sft_model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_new_tokens=64,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

            # Создание маски внимания для сгенерированных последовательностей
            gen_attention_mask = (outputs != tokenizer.eos_token_id).float().to(device)

            # Вычисление наград
            with torch.no_grad():
                rewards = reward_model(outputs, gen_attention_mask).squeeze()

            # Обновление baseline
            baseline = baseline_alpha * baseline + (1 - baseline_alpha) * rewards.mean().item()

            # Вычисление логарифмов вероятностей
            logits = sft_model(outputs, attention_mask=gen_attention_mask).logits
            log_probs = torch.log_softmax(logits, dim=-1)

            # Подготовка индексов для сбора вероятностей
            shifted_outputs = outputs[:, 1:].unsqueeze(-1)  # [batch, seq_len-1, 1]

            # Собираем вероятности только для реально сгенерированных токенов
            gathered_log_probs = log_probs[:, :-1].gather(
                dim=-1,
                index=shifted_outputs
            ).squeeze(-1)  # [batch, seq_len-1]

            # Вычисление потерь с правильным согласованием размерностей
            advantages = rewards - baseline
            loss = -(advantages.unsqueeze(1) * gathered_log_probs).mean()

            # Шаг оптимизации
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

reinforce(sft_model, reward_model, train_dataset, epochs=3)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [15]:
# Оценка после обучения
print("\nОценка после REINFORCE:")
aligned_reward = evaluate_reward(sft_model, reward_model, val_dataset)
print(f"Средняя награда: {aligned_reward:.3f}")

# Сравнение результатов
if aligned_reward > sft_reward:
    print(f"Улучшение: +{aligned_reward - sft_reward:.3f}")
else:
    print(f"Ухудшение: {aligned_reward - sft_reward:.3f}")


Оценка после REINFORCE:


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Средняя награда: -31.700
Улучшение: +3.015


In [17]:
# В функции reinforce добавьте:
sft_model.to(device)  # Перенос модели на устройство
reward_model.to(device)

RewardModel(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(49152, 576, padding_idx=2)
      (layers): ModuleList(
        (0-29): 30 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=576, out_features=576, bias=False)
            (k_proj): Linear(in_features=576, out_features=192, bias=False)
            (v_proj): Linear(in_features=576, out_features=192, bias=False)
            (o_proj): Linear(in_features=576, out_features=576, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
            (up_proj): Linear(in_features=576, out_features=1536, bias=False)
            (down_proj): Linear(in_features=1536, out_features=576, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        )